In [1]:
import torch
import numpy as np

In [2]:
# institate testing vectors
size = 10000
mask_size = int(np.round(size * size * 0.01))
x = torch.rand((size, 1))
y = torch.rand((1, size))

In [2]:
def compute_mask(w, topk_percentage):
    threshold = np.quantile(w.reshape(-1), topk_percentage)
    mask = np.where(w >= threshold)
    return mask

In [4]:
m = x.mm(y)
mask = compute_mask(m, 0.99)

In [19]:
def masked_outer_product(x, y, mask):
    values = torch.zeros(len(mask[0]))
    x = x.reshape(-1)
    y = y.reshape(-1)
    for i in range(len(mask[0])):
        values[i] = x[mask[0][i]] * y[mask[1][0]]
    
    return torch.sparse_coo_tensor(mask, values, (len(x), len(y)))
        

In [30]:
%timeit masked_outer_product(x, y, mask)

8.89 s ± 97.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit m = x.mm(y)
# torch.sparse_coo_tensor(mask, m[mask], m.shape)

132 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Benchmarkig torch.sparse vs elementwise masking

In [3]:
size = 128
mat1 = torch.rand((size, size))
vec_mask = compute_mask(mat1, 0.99)
vector = torch.rand((size, 1))
mat1_mask = torch.zeros_like(mat1)
sparse_mat1 = torch.sparse_coo_tensor(vec_mask, mat1[vec_mask], mat1.shape)
sparse_mat2 = torch._sparse_csr_tensor(torch.IntTensor(vec_mask[0]), torch.IntTensor(vec_mask[1]), mat1[vec_mask])

In [4]:
%timeit mat1_mask = torch.zeros_like(mat1)
%timeit mat1_mask[vec_mask] = 1
%timeit (mat1 * mat1_mask).mm(vector)

3.43 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
12.3 µs ± 96.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.7 µs ± 68.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
%timeit torch.sparse_coo_tensor(vec_mask, mat1[vec_mask], mat1.shape)
%timeit sparse_mat1.mm(vector)

76.6 µs ± 515 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
8.18 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%timeit torch._sparse_csr_tensor(torch.IntTensor(vec_mask[0]), torch.IntTensor(vec_mask[1]), mat1[vec_mask])
%timeit sparse_mat2.mm(vector)

36.2 µs ± 458 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


RuntimeError: Cannot access data pointer of Tensor that doesn't have storage

Making sparse matrices is really costly, but once they are made multiplying with them is probably faster than masking elementwise.

# Testing sparse out by blowing up both vectors

In [30]:
size = 1000
x = torch.rand(size)
y = torch.rand(size)
m = x.reshape((-1,1)).mm(y.reshape(1, -1))
vec_mask = compute_mask(m, 0.99)
m_mask = torch.zeros_like(m)
m_mask[vec_mask] = 1

In [22]:
x_mat = x.repeat((size, 1))

In [23]:
x_mat.mul(y)

tensor([[0.7931, 0.5761, 0.1189,  ..., 0.0296, 0.0773, 0.0324],
        [0.7931, 0.5761, 0.1189,  ..., 0.0296, 0.0773, 0.0324],
        [0.7931, 0.5761, 0.1189,  ..., 0.0296, 0.0773, 0.0324],
        ...,
        [0.7931, 0.5761, 0.1189,  ..., 0.0296, 0.0773, 0.0324],
        [0.7931, 0.5761, 0.1189,  ..., 0.0296, 0.0773, 0.0324],
        [0.7931, 0.5761, 0.1189,  ..., 0.0296, 0.0773, 0.0324]])

In [32]:
torch.outer(x, y, out = m_mask)

tensor([[0.4847, 0.4041, 0.3672,  ..., 0.5377, 0.4112, 0.1065],
        [0.7746, 0.6457, 0.5868,  ..., 0.8593, 0.6572, 0.1702],
        [0.2605, 0.2172, 0.1973,  ..., 0.2890, 0.2210, 0.0572],
        ...,
        [0.4132, 0.3444, 0.3130,  ..., 0.4583, 0.3505, 0.0908],
        [0.5005, 0.4173, 0.3792,  ..., 0.5553, 0.4246, 0.1100],
        [0.8659, 0.7218, 0.6559,  ..., 0.9606, 0.7346, 0.1902]])